In [1]:
import pandas as pd
import numpy as np
from persiantools.jdatetime import JalaliDate
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
Data=pd.read_csv('D:\Mohammad\Datasets\Dataset.csv')

df=pd.DataFrame(data=Data)

In [6]:
df.head()

,Unnamed: 0,CustomerID,OrderID,Date,Revenue,Date_per,per_month
0,0,28448640,32937314,20190321,5100,1398-01-01,1398-01
1,1,11253773,32934605,20190321,5100,1398-01-01,1398-01
2,2,11257623,32943465,20190321,11706,1398-01-01,1398-01
3,3,11012874,32941555,20190321,5100,1398-01-01,1398-01
4,4,10686656,32944079,20190321,5100,1398-01-01,1398-01


In [4]:
df['Date']=df['Date'].astype(str)
df['Date_per'] = df['Date'].apply(lambda x: JalaliDate.to_jalali(int(x[:4]),int(x[4:6]),int(x[6:])))
#df['per_date'] = JalaliDate(df['Date'])

In [5]:
#df['Date_per'] = JalaliDate(df['Date'])
df['per_month']=df['Date_per'].apply(lambda x: x.strftime('%Y-%m'))

In [7]:
Count_OrderID=df.groupby(['per_month','CustomerID'],as_index=False)['OrderID'].count()
Count_OrderID

,per_month,CustomerID,OrderID
0,1398-01,9590906,1
1,1398-01,9591119,1
2,1398-01,9591402,1
3,1398-01,9591831,1
4,1398-01,9592655,1
...,...,...,...
227257,1399-01,42938426,1
227258,1399-01,44137552,1
227259,1399-01,45409041,1
227260,1399-01,46380582,1


In [9]:
len(Count_OrderID['CustomerID'].unique())

179680

In [10]:
Count_OrderID.groupby(['per_month'],as_index=False)['OrderID'].sum()

,per_month,OrderID
0,1398-01,3279
1,1398-02,3503
2,1398-03,14380
3,1398-04,16228
4,1398-05,28663
5,1398-06,18630
6,1398-07,26699
7,1398-08,33233
8,1398-09,30989
9,1398-10,26642


In [11]:
Tot_Reven=df.groupby(['per_month'],as_index=False)['Revenue'].sum()
Tot_Reven

,per_month,Revenue
0,1398-01,19618499
1,1398-02,23860379
2,1398-03,91363493
3,1398-04,107350424
4,1398-05,146502637
5,1398-06,104375481
6,1398-07,150623504
7,1398-08,183010916
8,1398-09,180390183
9,1398-10,177407408


# Which month has the highest order and revenue ?

In [12]:
a=np.max(Tot_Reven)
b=np.max(Count_OrderID)
print(a)

print('*******************')

print(b)


per_month      1399-01
Revenue      183010916
dtype: object
*******************
per_month      1399-01
CustomerID    46549548
OrderID             45
dtype: object


# RFM Model          

# R = Recency

# F = Frequency

# M = Monetary

In [17]:
df_recency = df.groupby(['CustomerID'],
                        as_index=False)['Date_per'].max()
df_recency.columns = ['CustomerName', 'LastPurchaseDate']
recent_date = df_recency['LastPurchaseDate'].max()
df_recency['Recency'] = df_recency['LastPurchaseDate'].apply(
    lambda x: (recent_date - x).days)
df_recency.head()

,CustomerName,LastPurchaseDate,Recency
0,465132,1398-09-09,141
1,465164,1398-12-09,51
2,465198,1399-01-25,6
3,465204,1398-08-12,168
4,465211,1398-11-02,88


In [14]:
Frequency_df=df.drop_duplicates().groupby(['CustomerID'],as_index=False)['Date_per'].count()
Frequency_df.columns=['CustomerName','Frequency']
Frequency_df.head()

,CustomerName,Frequency
0,465132,1
1,465164,1
2,465198,1
3,465204,1
4,465211,1


In [15]:
RFM_Model = df_recency.merge(Frequency_df, on='CustomerName')
RFM_Model
# Segment Customers 

,CustomerName,LastPurchaseDate,Recency,Frequency
0,465132,1398-09-09,141,1
1,465164,1398-12-09,51,1
2,465198,1399-01-25,6,1
3,465204,1398-08-12,168,1
4,465211,1398-11-02,88,1
...,...,...,...,...
179675,46325082,1398-02-26,339,1
179676,46380582,1399-01-09,22,1
179677,46518839,1398-12-11,49,1
179678,46538916,1399-01-20,11,1
